In [1]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from scipy import stats
import numpy as np
import folium 
from folium import plugins

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

import mapboxgl
import os
from mapboxgl.viz import *
import os
from mapboxgl.utils import create_color_stops
import geopandas as gpd
import dash_table
from collections import OrderedDict

C:\Users\painc\Anaconda3\envs\iitp_keras\lib\site-packages\dash_bootstrap_components\_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-2cad0e5eb45e>:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-1-2cad0e5eb45e>:24: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


# 필요 함수 생성

In [2]:
data_path ='../data/'
# Json 파일 읽어오기
def read_json(jsonPath ,mod = 'r',encoding ='utf-8'):
    with open(jsonPath, mod, encoding = encoding) as common:
        config = json.load(common)
    return config
config = read_json(data_path + "json/config.json")

def transColName(cols):
    cols = cols.replace(' (%)', '')
    cols = cols.replace(' (명)', '')
    cols = cols.replace('[명]', '')
    cols = cols.replace('[백분율]', '')
    cols = cols.replace(')', '')
    cols = cols.replace('(', '_')
    cols = cols.replace(' ', '_')
    cols = cols.replace('1', '')
    return cols 


# 필요 데이터 Import

In [3]:
# 시도 데이터
prePredict = pd.read_csv(config['data_path']+'csv/cleaned/merged_with_sido.csv').iloc[:,1:]
prePredict=prePredict.fillna(0)

# 범죄 예측 데이터
df_final= pd.read_csv(config['data_path']+'csv/cleaned/crime_predicted_xgboost_mean_scaled.csv')
df_final=df_final[df_final.년도 <= 2025 ]

# 년도별 위험도 분석
df_danger_final = pd.read_csv(config['data_path']+'csv/cleaned/crime_increase_or_not_data.csv')
df_danger_final=df_danger_final[df_danger_final.년도 <= 2025 ]

In [4]:
with open(config['data_path']+'geojson/korea_geojson2.geojson', encoding='UTF-8') as f:
    data = json.load(f)
# df = 시도 코드


indi_df = pd.concat(
    [
        prePredict.iloc[:,0:2],# 시도 및 년도
        prePredict.iloc[:,15:43] # 종속변수
        ],
        axis=1
    )
# 시도, 년도, 범죄, 시도코드
crime_df = pd.concat(
    [
        prePredict.iloc[:,0:2],# 시도 및 년도
        prePredict.iloc[:,2:15],# 범죄
        prePredict.iloc[:,-1:]# 시도코드 가져오기
        ],
        axis=1
        )

mapper = [
    ('경기', '경기도'),
    ('서울', '서울특별시'),
    ('충북', '충청북도'),
    ('인천', '인천광역시'),
    ('충남', '충청남도'),
    ('광주', '광주광역시'),
    ('부산', '부산광역시'),
    ('강원', '강원도'),
    ('전남', '전라남도'),
    ('대구', '대구광역시'),
    ('전북', '전라북도'),
    ('울산', '울산광역시'),
    ('제주', '제주특별자치도'),
    ('경북', '경상북도'),
    ('세종', '세종특별자치시'),
    ('경남', '경상남도'),
    ('대전', '대전광역시')
]
get_region = lambda gubun: [x[1] for x in mapper if x[0] == gubun][0]

crime_df['시도'] = crime_df.시도.apply(get_region)
crime_df = crime_df.rename(columns={"시도":"CTP_KOR_NM"})

indi_df['시도'] = indi_df.시도.apply(get_region)
indi_df = indi_df.rename(columns={"시도":"CTP_KOR_NM"})

# 위험도 데이터
df_danger_final['시도'] = df_danger_final.시도.apply(get_region)
df_danger_final = df_danger_final.rename(columns={"시도":"CTP_KOR_NM"})

# 예측 데이터
df_final['시도'] = df_final.시도.apply(get_region)
df_final = df_final.rename(columns={"시도":"CTP_KOR_NM"})



In [5]:
#데이터 테이블 범죄
table=dash_table.DataTable(
    data=crime_df.to_dict('records'),
    columns=[{'id': c, 'name': c} for c in crime_df.columns],
            page_size=20,
    fixed_rows={'headers': True},
            style_table={'overflowX': 'auto'},
    style_cell={
        # all three widths are needed
        'minWidth': '180px', 'width': '180px', 'maxWidth': '180px',
        'overflow': 'hidden',
        'textOverflow': 'ellipsis',
    },
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
)
#데이터 테이블 민간 요소
table_1=dash_table.DataTable(
    data=indi_df.to_dict('records'),
    columns=[{'id': c, 'name': c} for c in indi_df.columns],
            page_size=20,
    fixed_rows={'headers': True},
            style_table={'overflowX': 'auto'},
    style_cell={
        # all three widths are needed
        'minWidth': '180px', 'width': '180px', 'maxWidth': '180px',
        'overflow': 'hidden',
        'textOverflow': 'ellipsis',
    },
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
)

drop_style = {'width': '150px','height': '50px','textAlign': 'center'}
#드랍박스 범죄
crime_drop =dcc.Dropdown(
                id = "crime-dropdown",
                options = [
                    {"label":i,"value":i} for i in crime_df.columns[2:15]
                ], style= drop_style
            )
            
#드랍박스 민간요소
personal_drop =dcc.Dropdown(
                id = "person-dropdown",
                options = [
                    {"label":i,"value":i} for i in indi_df.columns[2:-1]
                ], style= drop_style
            )
#드랍박스 시도
country_drop =dcc.Dropdown(
                id = "country-dropdown",
                options = [
                    {"label":i,"value":i} for i in crime_df.CTP_KOR_NM.unique()
                ], style= drop_style
            )

year_marks = {}
for i in range(2002,2020,1):
    year_marks[i] = f'{i}년'

year_range_slider = dcc.RangeSlider(
            id='year-slider',
            min=2002,
            max=2019,
            step=None,
            marks=year_marks,
            value=[2002, 2019],            
        )

predict_year_marks = {}
for i in range(2002,2026,1):
    predict_year_marks[i] = f'{i}년'

predict_year_range_slider = dcc.RangeSlider(
            id='predict_year-slider',
            min=2002,
            max=2025,
            step=None,
            marks=predict_year_marks,
            value=[2002, 2023],            
        )
        
#지도 그리기
map=dcc.Graph(
    id = "crime-map",
    style= {'width': '100%','height': '100%'}
    )
#분석결과
main_all=dbc.Card([
            dbc.CardBody([
                html.H2('분석 리서치', style= {'textAlign':'center','align': 'center',
                       'width':'100%','height':'50',"margin-left": "1rem"}),
                # 연도 표시
                predict_year_range_slider,
                dbc.Row([
                country_drop, crime_drop
                ]),
                dbc.Row(style={'textAlign':'center','align': 'center',
                       'width':'100%','height':'200%',"margin-left": "1rem"},
                     children=[
               dcc.Graph(id = "line_final")
                ]),
                dbc.Row(style={'textAlign':'center','align': 'center',
                       'width':'100%','height':'200%',"margin-left": "1rem"},
                     children=[
                dbc.Col(style={'textAlign':'center','align': 'center',
                       'width':'50%','height':'200%'},
                     children=[ 
                    dcc.Graph(id = "crime-map_2"),   
                ]),
                dbc.Col(style={'textAlign':'center',
                       'width':'50%','height':'50%'},
                     children=[
                         dcc.Graph(id = "crime-bar_2"),
                         dcc.Graph(id = "crime-pie_2")
                     ]),
                ]),
            ])
        ])

#범죄 코드
main =dbc.Card([
        dbc.CardBody([
            html.H2('지역별 Crime EDA', style= {'textAlign':'center','align': 'center',
                       'width':'100%','height':'50',"margin-left": "1rem"}),
            # 연도 표시
            year_range_slider,
            dbc.Row([
                country_drop, crime_drop
            ]),
            
            dbc.Row(style={'textAlign':'center','align': 'center',
                       'width':'100%','height':'200%',"margin-left": "1rem"},
                     children=[
               dcc.Graph(id = "line")
            ]),
            html.H2('연도별 Crime EDA', style= {'textAlign':'center','align': 'center',
                       'width':'100%','height':'50',"margin-left": "1rem"}),
            dbc.Row(style={'textAlign':'center','align': 'center',
                       'width':'100%','height':'100%'},
                     children=[
                dbc.Col(style={'textAlign':'center','align': 'center',
                       'width':'50%','height':'200%'},
                     children=[ 
                    dcc.Graph(id = "crime-map"),   
                ]),

                dbc.Col(style={'textAlign':'center',
                       'width':'50%','height':'50%'},
                     children=[
                    dcc.Graph(id = "crime-pie"),
                    dcc.Graph(id = "crime-bar")
                ])
            ])
        ])
    ])
#민간 요소 코드
main_1 =dbc.Card([
    dbc.CardBody(style={'textAlign':'center','align': 'center',
                       'width':'100%','height':'100%',"margin-left": "1rem"},
                 children=[
        html.H2('지역별 민간요소 EDA', style= {'textAlign': 'center',}),
        # 연도 표시
        year_range_slider,
        dbc.Row([
            country_drop, personal_drop
        ]),
        dbc.Row(style={'textAlign':'center','align': 'center',
                       'width':'100%','height':'100%',"margin-left": "1rem"},
                 children=[
           dcc.Graph(id = "line_1")
        ]),
        html.H2('연도별 민간요소 EDA', style= {'textAlign': 'center'}),
        dbc.Row(style={'textAlign':'center','align': 'center',
                       'width':'100%','height':'100%'},
                 children=[
            dbc.Col(style={'textAlign':'center','align': 'center',
                       'width':'50%','height':'200%'},
                 children=[ 
                dcc.Graph(id = "crime-map_1"),   
            ]),

            dbc.Col(style={'textAlign':'center',
                       'width':'50%','height':'50%'},
                 children=[
                dcc.Graph(id = "crime-pie_1"),
                dcc.Graph(id = "crime-bar_1")
            ]),
        ]),
       
    ])
])

In [6]:
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem", 
    "background-color": "#f8f9fa",
}
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

sidebar = html.Div(
    [
        
        html.Img(src=app.get_asset_url('./기쁨.png'),
            style={'width':'230px','height':'160px'}),
        html.Hr(),
        html.P(
            "Category", className="lead"
        ),
        dbc.Nav(
            [
                dbc.NavLink("범죄", href="/", active="exact"),
                dbc.NavLink("민간요소", href="/page-1", active="exact"),
                dbc.NavLink("분석결과", href="/page-2", active="exact"),
        
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id="page-content", style=CONTENT_STYLE)

app.layout = html.Div([dcc.Location(id="url"), sidebar, content])



In [7]:
@app.callback(
    Output("page-content", "children"), 
    [Input("url", "pathname")])
def render_page_content(pathname):
    if pathname == "/":
        return html.P([main,table])
    elif pathname == "/page-1":
        return html.P([main_1,table_1])
    elif pathname == "/page-2":
        year_marks = {}
        for i in range(2002,2026,1):
            year_marks[i] = f'{i}년'
        return html.P([main_all])

    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )

@app.callback(
    Output("line", "figure"),
    Input("crime-dropdown", "value"),
    Input("country-dropdown","value"),
    Input("year-slider","value")    
)
def line_graph(crime,country,years_):
    dt0 = crime_df.loc[crime_df.CTP_KOR_NM==country,:]
    dt0 = dt0[(dt0.년도 >= years_[0])&(dt0.년도 <= years_[1])]
    fig0 = px.line(dt0, x=dt0.년도.unique(), y=dt0[crime],height=500, width=1850)
    fig0.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",)
    
    return fig0

@app.callback(
    Output("crime-map", "figure"),
    Output("crime-pie", "figure"),
    Output("crime-bar", "figure"),
    Input("crime-dropdown","value"),
    Input("year-slider","value")    
    
)
def update_graph(crime,years_):
    dt = crime_df.loc[crime_df.년도==years_[1],:]
    geo_df = gpd.GeoDataFrame.from_features(
        data["features"]
    ).merge(dt, on="CTP_KOR_NM").set_index("CTP_KOR_NM")

    fig = px.choropleth(geo_df,
                       geojson=geo_df.geometry,
                       locations=geo_df.index,
                       color=crime,
                       projection="mercator",
                       height=900)
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",)
    
    colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']
    fig1 = go.Figure(data=[go.Pie(labels=dt.CTP_KOR_NM,
                             values=dt[crime])])
    fig1.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#444444', width=1)))
    fig1.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",)
    fig2 = px.bar(crime_df, x=dt.CTP_KOR_NM, y=dt[crime], color=dt[crime])
    fig2.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",)
    return fig,fig1,fig2

@app.callback(
    Output("line_1", "figure"),
    Input("person-dropdown", "value"),
    Input("country-dropdown","value"),
    Input("year-slider","value")    
)
def line_graph(person,country,year):
    dt0 = indi_df.loc[indi_df.CTP_KOR_NM==country,:]
    dt0 = dt0[(dt0.년도 >= year[0])&(dt0.년도 <= year[1])]
    fig0 = px.line(dt0, x=dt0.년도.unique(), y=dt0[person],height=500, width=1850 )
    fig0.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",)
    
    return fig0

@app.callback(
    Output("crime-map_1", "figure"),
    Output("crime-pie_1", "figure"),
    Output("crime-bar_1", "figure"),
    Input("year-slider","value"),
    Input("person-dropdown","value")
)
def update_graph(year,person):
    dt1 = indi_df.loc[indi_df.년도==year[1],:]
    geo_df = gpd.GeoDataFrame.from_features(
        data["features"]
    ).merge(dt1, on="CTP_KOR_NM").set_index("CTP_KOR_NM")

    fig = px.choropleth(geo_df,
                       geojson=geo_df.geometry,
                       locations=geo_df.index,
                       color=person,
                       projection="mercator",
                       height=900)
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",)
    
    colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']
    fig1 = go.Figure(data=[go.Pie(labels=dt1.CTP_KOR_NM,
                             values=dt1[person])])
    fig1.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#444444', width=1)))
    fig1.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",)
    fig2 = px.bar(indi_df, x=dt1.CTP_KOR_NM, y=dt1[person],  color=dt1[person])
    fig2.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",)
    return fig,fig1,fig2

## 최종 결과
@app.callback(
    Output("line_final", "figure"),
    Input("crime-dropdown", "value"),
    Input("country-dropdown","value"),
    Input("predict_year-slider","value")    
)
def line_graph(crime,country,years_):
    dt0 = df_final.loc[df_final.CTP_KOR_NM==country,:]
    dt0 = dt0[(dt0.년도 >= years_[0])&(dt0.년도 <= years_[1])]
    fig0 = px.line(dt0, x=dt0.년도.unique(), y=dt0[crime],height=500, width=1850)
    fig0.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",)
    
    return fig0

@app.callback(
    Output("crime-map_2", "figure"),
    Output("crime-pie_2", "figure"),
    Output("crime-bar_2", "figure"),
    Input("crime-dropdown","value"),
    Input("predict_year-slider","value")    
    
)
def update_graph(crime,years_):    
    dt = df_final.loc[df_final.년도==years_[1],:]
    dt_1 = df_danger_final.loc[df_danger_final.년도==years_[1],:]
    geo_df = gpd.GeoDataFrame.from_features(
        data["features"]
    ).merge(dt_1, on="CTP_KOR_NM").set_index("CTP_KOR_NM")

    fig = px.choropleth(geo_df,
                       geojson=geo_df.geometry,
                       locations=geo_df.index,
                       color=crime,
                       projection="mercator",
                       height=900)

    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",)
    
    colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

    fig1 = go.Figure(data=[go.Pie(labels=dt.CTP_KOR_NM,
                             values=dt[crime])])
    fig1.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#444444', width=1)))
    fig1.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",)
    fig2 = px.bar(df_final, x=dt.CTP_KOR_NM, y=dt[crime], color=dt[crime])
    fig2.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",)
    return fig,fig1,fig2

In [8]:
if __name__ == "__main__":
    app.run_server(port=1211)

Dash is running on http://127.0.0.1:1211/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:1211/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Oct/2021 17:30:07] "GET /page-2 HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2021 17:30:08] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2021 17:30:08] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2021 17:30:08] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2021 17:30:08] "GET /assets/기쁨.png HTTP/1.1" 404 -
127.0.0.1 - - [19/Oct/2021 17:30:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2021 17:30:08] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2021 17:30:08] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2021 17:30:08] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2021 17:30:08] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\painc\Anaconda3\envs\iitp_keras\lib\site-packages\pandas\core\indexes\base.py", line 3361, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 76, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: None

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\painc\Anaconda3\envs\iitp_keras\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\painc\Anaconda3\envs\iitp_keras\lib\site-packages\flask\app.py", line 1518

127.0.0.1 - - [19/Oct/2021 17:30:08] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\painc\Anaconda3\envs\iitp_keras\lib\site-packages\pandas\core\indexes\base.py", line 3361, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 76, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: None

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\painc\Anaconda3\envs\iitp_keras\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\painc\Anaconda3\envs\iitp_keras\lib\site-packages\flask\app.py", line 1518

...

127.0.0.1 - - [19/Oct/2021 17:30:18] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\painc\Anaconda3\envs\iitp_keras\lib\site-packages\pandas\core\indexes\base.py", line 3361, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 76, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: None

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\painc\Anaconda3\envs\iitp_keras\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\painc\Anaconda3\envs\iitp_keras\lib\site-packages\flask\app.py", line 1518

127.0.0.1 - - [19/Oct/2021 17:30:18] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\painc\Anaconda3\envs\iitp_keras\lib\site-packages\pandas\core\indexes\base.py", line 3361, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 76, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: None

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\painc\Anaconda3\envs\iitp_keras\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\painc\Anaconda3\envs\iitp_keras\lib\site-packages\flask\app.py", line 1518

127.0.0.1 - - [19/Oct/2021 17:30:19] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [19/Oct/2021 17:30:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2021 17:30:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2021 17:30:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2021 17:30:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2021 17:30:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2021 17:30:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2021 17:30:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2021 17:30:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2021 17:30:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2021 17:30:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2021 17:30:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.